In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from PIL import Image
# Parameters of dynamo
h = 1.0   # Some constant value for h
alpha_0 = 1.0  # Some constant value for alpha_0
q = 1.0   # Some constant value for q
Omega = 1.0  
eta_t = 1.0  

#parameters for program
L = 10.0  
N = 100   
T = 20.0   
dt = 0.01 

# Spatial grid
dz = 2*L / (N - 1)
z = np.linspace(-L, L, N)

# Initial conditions
def initial_condition(x):
    return 50000 * np.sin(np.pi *(x+1)/2)
B_r0 = np.zeros(N)
B_phi0 = np.zeros(N)
B_r0=initial_condition(z)
B_phi0=initial_condition(z)

# Function to calculate alpha at each spatial point
def alpha(z):
    return np.sin(np.pi * z / 2)

# Function to calculate derivatives
def dBr_dz(B_r):
    return np.gradient(B_r, dz)

def d2Br_dz2(B_r):
    return np.gradient(np.gradient(B_r, dz), dz)

def d2Bphi_dz2(B_phi):
    return np.gradient(np.gradient(B_phi, dz), dz)

# Right-hand side functions for the differential equations
def dBrdt(B_r, B_phi):
    return - (h * alpha_0 / eta_t) * dBr_dz(B_phi * alpha(z)) + d2Br_dz2(B_r)

def dBphidt(B_r, B_phi):
    return - (h**2 * q * Omega / eta_t) * B_r + d2Bphi_dz2(B_phi)

# Fourth-order Runge-Kutta method
def rk4_step(B_r, B_phi):
    k1_r = dt * dBrdt(B_r, B_phi)
    k1_phi = dt * dBphidt(B_r, B_phi)
    
    k2_r = dt * dBrdt(B_r + 0.5 * k1_r, B_phi + 0.5 * k1_phi)
    k2_phi = dt * dBphidt(B_r + 0.5 * k1_r, B_phi + 0.5 * k1_phi)
    
    k3_r = dt * dBrdt(B_r + 0.5 * k2_r, B_phi + 0.5 * k2_phi)
    k3_phi = dt * dBphidt(B_r + 0.5 * k2_r, B_phi + 0.5 * k2_phi)
    
    k4_r = dt * dBrdt(B_r + k3_r, B_phi + k3_phi)
    k4_phi = dt * dBphidt(B_r + k3_r, B_phi + k3_phi)
    
    B_r_new = B_r + (k1_r + 2 * k2_r + 2 * k3_r + k4_r) / 6
    B_phi_new = B_phi + (k1_phi + 2 * k2_phi + 2 * k3_phi + k4_phi) / 6
    
    return B_r_new, B_phi_new

def update_plot(frame):
    global B_r0, B_phi0
    for _ in range(100):
        B_r0, B_phi0 = rk4_step(B_r0, B_phi0)
    line_br.set_ydata(B_r0)
    line_bphi.set_ydata(B_phi0)
    ax.set_title(f'Time = {frame * 100 * dt:.2f}')
    return line_br, line_bphi
    
fig, ax = plt.subplots()
ax.set_xlim(-L, L)
ax.set_xlabel('z')
ax.set_ylabel('Magnetic Field')
ax.set_title('Magnetic Field Evolution')


line_br, = ax.plot(z, B_r0, label='$B_r$')
line_bphi, = ax.plot(z, B_phi0, label='$B_{\phi}$')
ax.legend()

ani = FuncAnimation(fig, update_plot, frames=int(T/(100*dt)), interval=50, blit=True)#this also doubles as the main loop
ani.save('magnetic_field_evolution.gif', writer='pillow')
plt.show()
